In [26]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


np.random.seed(0)
n = 15
x = np.linspace(0,10,n) + np.random.randn(n)/5
y = np.sin(x)+x/6 + np.random.randn(n)/10


X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

def part1_scatter():
    import matplotlib.pyplot as plt
    %matplotlib notebook
    plt.figure()
    plt.scatter(X_train, y_train, label='training data')
    plt.scatter(X_test, y_test, label='test data')
    plt.legend(loc=4);
    
    


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
def answer_one():
    x_1 = x.reshape(-1,1)  # change x into columns vector
    y_1 = y.reshape(-1,1)
    result = []
    for i in [1,3,6,9]:
        poly = PolynomialFeatures(degree = i )
        x_p = poly.fit_transform(x_1)
        X_train, X_test, y_train, y_test = train_test_split( x_p, y ,random_state =0 )
        linreg = LinearRegression().fit(X_train, y_train)
        to_predict = poly.fit_transform(np.linspace(0,10,100).reshape(-1,1))
        result.append(linreg.predict(to_predict).reshape(1,100)[0])
    result=np.array(result)
    return result
answer_one().shape

(4, 100)

In [18]:

def plot_one(degree_predictions):
    import matplotlib.pyplot as plt
    %matplotlib notebook
    plt.figure(figsize=(10,5))
    plt.plot(X_train, y_train, 'o', label='training data', markersize=10)
    plt.plot(X_test, y_test, 'o', label='test data', markersize=10)
    for i,degree in enumerate([1,3,6,9]):
        plt.plot(np.linspace(0,10,100), degree_predictions[i], alpha=0.8, lw=2, label='degree={}'.format(degree))
    plt.ylim(-1,2.5)
    plt.legend(loc=4)

#plot_one(answer_one())

In [19]:
def answer_two():
    from sklearn.linear_model import LinearRegression
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.metrics.regression import r2_score
    r2_train = []
    r2_test = []
    for i in range(10):
        poly = PolynomialFeatures(degree = i)
        x_p = poly.fit_transform(X_train.reshape(-1,1))
        x_p_t = poly.fit_transform(X_test.reshape(-1,1))
        linreg = LinearRegression().fit(x_p,y_train.reshape(-1,1))
        r2_train.append(linreg.score(x_p,y_train.reshape(-1,1)))
        r2_test.append(linreg.score(x_p_t, y_test.reshape(-1,1)))
    return (r2_train,r2_test)
answer_two()

([0.0,
  0.42924577812346632,
  0.4510998044408247,
  0.58719953687798476,
  0.91941944717693158,
  0.97578641430681867,
  0.99018233247950882,
  0.99352509278402779,
  0.99637545387745396,
  0.99803706256582003],
 [-0.47808641737141788,
  -0.45237104233936676,
  -0.068569841499158901,
  0.0053310529457279987,
  0.73004942818715057,
  0.87708300916504855,
  0.9214093981277145,
  0.92021504118071573,
  0.63247943762578418,
  -0.64525446868451841])

In [20]:
'''%matplotlib notebook
import matplotlib.pyplot as plt
plt.figure()
plt.plot(answer_two()[0],'r-')
plt.plot(answer_two()[1],'b-')'''
def answer_three():
    return (0,9,7)
answer_three()

(0, 9, 7)

In [31]:
def answer_four():
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.linear_model import Lasso, LinearRegression
    from sklearn.metrics.regression import r2_score
    poly = PolynomialFeatures(degree = 12)
    X_p = poly.fit_transform(X_train.reshape(-1,1))
    X_p_t = poly.fit_transform(X_test.reshape(-1,1))
    linreg = LinearRegression().fit(X_p,y_train.reshape(-1,1))
    lassoreg = Lasso(alpha = 0.01, max_iter = 10000).fit(X_p,y_train.reshape(-1,1))
    LinearRegression_R2_test_score = linreg.score(X_p_t,y_test.reshape(-1,1))
    Lasso_R2_test_score = lassoreg.score(X_p_t,y_test.reshape(-1,1))
    return (LinearRegression_R2_test_score, Lasso_R2_test_score)
answer_four()

/opt/conda/lib/python3.5/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


(-4.3119904517942373, 0.8406625614750235)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


mush_df = pd.read_csv('mushrooms.csv')
mush_df2 = pd.get_dummies(mush_df)

X_mush = mush_df2.iloc[:,2:]
y_mush = mush_df2.iloc[:,1]

# use the variables X_train2, y_train2 for Question 5
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_mush, y_mush, random_state=0)


X_subset = X_test2
y_subset = y_test2

In [43]:
def answer_five():
    from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier(random_state=0).fit(X_train2,y_train2)
    f_i = pd.Series(clf.feature_importances_,index = list(X_train2.columns))
    f_i.sort_values(ascending =False, inplace = True)
    return list(f_i.index[:5])
answer_five()

['odor_n', 'stalk-root_c', 'stalk-root_r', 'spore-print-color_r', 'odor_l']

In [51]:
def answer_six():
    from sklearn.svm import SVC
    from sklearn.model_selection import validation_curve
    param_range = np.logspace(-4, 1, 6)
    train_scores, test_scores = validation_curve(SVC(kernel='rbf', C=1, random_state = 0), X_subset, y_subset,
                                            param_name='gamma',
                                            param_range=param_range, cv=3, scoring = "accuracy")
    training_scores = train_scores.mean(axis=1)
    test_scores = test_scores.mean(axis=1)
    return (training_scores, test_scores)
answer_six()

(array([ 0.56647847,  0.93155951,  0.99039881,  1.        ,  1.        ,  1.        ]),
 array([ 0.56768547,  0.92959558,  0.98965952,  1.        ,  0.99507994,
         0.52240279]))

In [1]:
'''%matplotlib notebook
import matplotlib.pyplot as plt
plt.figure()
plt.plot([ 0.56647847,  0.93155951,  0.99039881,  1.        ,  1.        ,  1.        ],'r-')
plt.plot([ 0.56768547,  0.92959558,  0.98965952,  1.        ,  0.99507994, 0.52240279],'b-')'''
def answer_seven():
    
    return (0.0001,10,0.1)
answer_seven()

(0.0001, 10, 0.1)